In [22]:
from __future__ import division
%matplotlib inline
import CameraNetwork
import CameraNetwork.global_settings as gs
from CameraNetwork.utils import extractThumbnails
import cPickle
import cv2
import datetime
import Image
import ImageDraw
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import skimage.io as io
import time
import tqdm
from tqdm import tqdm_notebook
import traceback
plt.rcParams['figure.figsize'] = (8.0, 8.0)

In [2]:
c = CameraNetwork.CLIclient(timeout=60)
proxy_params = CameraNetwork.retrieve_proxy_parameters()
c.start(proxy_params)
time.sleep(3)
print c.client_instance.servers

ERROR:root:Failed retrieving proxy parameters:
Traceback (most recent call last):
  File "i:\amit\office backup\amit\studies\cameranetwork_git\CameraNetwork\internet.py", line 56, in retrieve_proxy_parameters
    response = urllib2.urlopen(req)
  File "C:\Anaconda2\lib\urllib2.py", line 154, in urlopen
    return opener.open(url, data, timeout)
  File "C:\Anaconda2\lib\urllib2.py", line 435, in open
    response = meth(req, response)
  File "C:\Anaconda2\lib\urllib2.py", line 548, in http_response
    'http', request, response, code, msg, hdrs)
  File "C:\Anaconda2\lib\urllib2.py", line 473, in error
    return self._call_chain(*args)
  File "C:\Anaconda2\lib\urllib2.py", line 407, in _call_chain
    result = func(*args)
  File "C:\Anaconda2\lib\urllib2.py", line 556, in http_error_default
    raise HTTPError(req.get_full_url(), code, msg, hdrs, fp)
HTTPError: HTTP Error 404: Not Found



['109', '109L', '110', '110L', '111', '111L', '112L', '113', '113L', '114', '114L', '115', '115L', '116', '116L', '118', '119', '120']


In [12]:
remote_servers = [server_id for server_id in c.servers_list if not server_id.endswith("L")]

In [31]:
def get_thumbs(client, server_id):
    days = client[server_id].days()[1][0]["days_list"]
    thumbnails = []
    dfs = []
    for day in days:
        time_ind = datetime.datetime(*[int(i) for i in day.split("_")])
        try:
            reply = c[server_id].thumbnails(time_ind.date(), "60T")
            thumbnails.extend(extractThumbnails(reply[1][0]["thumbnails"]))
            dfs.append(reply[1][0]["thumbs_df"])
        except Exception, e:
            print(server_id, day, type(e))
    
    if dfs:
        df = pd.concat(dfs)
    else:
        df = None

    return server_id, thumbnails, df, dfs

In [32]:
t0 = time.time()
replies = Parallel(backend="threading", n_jobs=len(remote_servers))(
    delayed(get_thumbs)(c, server_id) for server_id in remote_servers)
t1 = time.time() - t0

('117', '2017_01_25', <type 'exceptions.Exception'>)
('119', '2017_03_20', <type 'exceptions.Exception'>)('115', '2017_04_06', <type 'exceptions.Exception'>)

('114', '2017_04_02', <type 'exceptions.Exception'>)('116', '2017_04_06', <type 'exceptions.Exception'>)

('111', '2017_04_03', <type 'exceptions.Exception'>)
('113', '2017_04_08', <type 'exceptions.Exception'>)
('117', '2017_04_19', <type 'exceptions.Exception'>)
('119', '2017_03_23', <type 'exceptions.Exception'>)
('117', '2017_04_20', <type 'exceptions.Exception'>)
('119', '2017_03_24', <type 'exceptions.Exception'>)
('119', '2017_03_25', <type 'exceptions.Exception'>)
('119', '2017_03_26', <type 'exceptions.Exception'>)
('119', '2017_03_27', <type 'exceptions.Exception'>)
('119', '2017_03_28', <type 'exceptions.Exception'>)
('119', '2017_03_29', <type 'exceptions.Exception'>)
('119', '2017_03_30', <type 'exceptions.Exception'>)
('119', '2017_03_31', <type 'exceptions.Exception'>)
('119', '2017_04_01', <type 'exceptions.Except

In [26]:
t1

512.4509999752045

In [51]:
with open("thumbs_remote.pkl", "wb") as f:
    cPickle.dump(replies, f)

In [56]:
new_dfs = {}
for server, thumbnails, df, _ in replies:
    if df is None:
        continue
            
    paths = []
    for i, (ind, row) in enumerate(df.iterrows()):
        thumbs_path = os.path.join("thumbnails", server, os.path.split(row["path"])[-1])
        thumbs_path = thumbs_path[:-3] + ".jpg"
        paths.append(thumbs_path)
        folder, _ = os.path.split(thumbs_path)
        if not os.path.exists(folder):
            os.makedirs(folder)
        io.imsave(thumbs_path, thumbnails[i])
    df["thumbnail"] = paths
    new_dfs[server] = df

In [58]:
with open("thumbnails_downloaded_remote.pkl", "wb") as f:
    cPickle.dump(new_dfs, f)

In [54]:
servers_df["109"]

KeyError: '109'